In [1]:
from naslib.utils.io import read_json
from pathlib import Path 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.axes import Axes

In [18]:
sns.set_style("darkgrid")
rcParams['axes.titlepad'] = 15
rcParams['font.size'] = 10.5

In [3]:
QUERY_KEY = "valid_acc"
LOG_FILENAME = "errors.json"

def collect_info_all_seeds(folder: Path, filename: str = LOG_FILENAME, query_key: str = QUERY_KEY):
    matches = list(folder.rglob(pattern=f"./seed=*"))

    scores={}
    for p in matches:
        scores[p.name] = read_json(p / filename)[query_key]
    df = pd.DataFrame(scores)
    df.index += 1   # python indexing starts from 0
    df.index.name = "epochs"
    return df.sort_index(axis=1).reset_index()

In [4]:
def plot_single_experiment(path: Path, ax: Axes, label: str):
    
    df_wide = collect_info_all_seeds(folder=path)
    df_long = pd.melt(df_wide, id_vars='epochs')
    sns.lineplot(x="epochs", data=df_long, y="value", err_style="band", errorbar="sd", label=label, ax=ax)
    return ax

In [5]:
# Parameters
dataset =  "cifar10" # "cifar10" , "cifar100", "ImageNet16-120"
acq_func = "its"
num_init = 20
num_quantiles= 10 #20

In [6]:
home = Path.home() / "Desktop/Experiments"

gaussian = home / "bananas__ensemble_mlp__gaussian" / "nasbench201" / dataset / f"acq={acq_func}" / f"num_init={num_init}"
scp_03 = home / "bananas__ensemble_mlp__CP_split" / "nasbench201" / dataset / f"acq={acq_func}__train_cal_split=03__num_quantiles={num_quantiles}" / f"num_init={num_init}"
scp_05 = home / "bananas__ensemble_mlp__CP_split" / "nasbench201" / dataset / f"acq={acq_func}__train_cal_split=05__num_quantiles={num_quantiles}" / f"num_init={num_init}"

label_and_path = {
    "Uncalibrated": gaussian,
    "SCP calibrated (70/30)": scp_03,
    "SCP calibrated (50/50)": scp_05
}

In [ ]:
fig, ax = plt.subplots(1,1)
plt.title("Comparative Performance")

for label, path in label_and_path.items():
    ax = plot_single_experiment(path=path, ax=ax, label=label)

plt.xlabel("epochs")
plt.ylabel("validation accuracy")
plt.legend(loc="lower right")
plt.tight_layout()